In [1]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
import os
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
from mtcnn import MTCNN
import cv2 as cv
from sklearn.model_selection import train_test_split
import mediapipe as mp
from scipy.spatial import distance
import time
from tensorflow.keras.models import load_model
from watchdog.observers import Observer
from watchdog.events import FileSystemEventHandler

In [2]:
# Inisialisasi MTCNN untuk deteksi wajah
detector = MTCNN()

In [3]:
def extract_face(image_path, required_size=(160, 160)):
    image = cv.imread(image_path)
    if image is None:
        print(f"⚠️ Failed to read image: {image_path}")
        return None
    image_rgb = cv.cvtColor(image, cv.COLOR_BGR2RGB)
    
    # Coba deteksi wajah dengan MTCNN
    results = detector.detect_faces(image_rgb)
    if len(results) > 0:
        x1, y1, width, height = results[0]['box']
        x1, y1 = abs(x1), abs(y1)
        x2, y2 = x1 + width, y1 + height
        x1, y1 = max(0, x1), max(0, y1)
        x2, y2 = min(image_rgb.shape[1], x2), min(image_rgb.shape[0], y2)
        face = image_rgb[y1:y2, x1:x2]
        face_resized = cv.resize(face, required_size)
        return face_resized
    
    # Jika MTCNN gagal, coba deteksi dengan Haarcascade
    gray = cv.cvtColor(image, cv.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, 1.3, 5)
    if len(faces) > 0:
        x, y, w, h = faces[0]
        face = image_rgb[y:y+h, x:x+w]
        face_resized = cv.resize(face, required_size)
        return face_resized
    
    print(f"⚠️ No face detected in: {image_path}")
    return None

In [4]:
def load_dataset(dataset_dir):
    X, y = [], []
    print(f"Scanning dataset directory: {dataset_dir}")
    for nim in os.listdir(dataset_dir):
        nim_path = os.path.join(dataset_dir, nim)
        if not os.path.isdir(nim_path):
            continue  
        print(f"Processing NIM: {nim}")
        for image_name in os.listdir(nim_path):
            image_path = os.path.join(nim_path, image_name)
            if not os.path.isfile(image_path):
                continue  
            print(f"Processing: {image_path}")
            face = extract_face(image_path)
            if face is None:
                print(f"⚠️ Face not detected in: {image_path}")
                continue
            X.append(face)
            y.append(nim)
    print(f"✅ Dataset loaded: {len(X)} images found")
    print(f"Sample X[0] shape: {X[0].shape if X else 'EMPTY'}")
    print(f"Sample y[0]: {y[0] if y else 'EMPTY'}")
    return np.array(X), np.array(y)

In [5]:
# Path dataset dan cascade (sesuai direktori Anda)
dataset_dir = 'C:/Users/jefta/OneDrive/Dokumen/Tugas Akhir Next js/prototype-leads/Backend/dataset_model_skripsi'
cascade_path = "C:/Users/jefta/OneDrive/Dokumen/Tugas Akhir Next js/prototype-leads/Backend/haarcascade_frontalface_default.xml"
face_cascade = cv.CascadeClassifier(cascade_path)

if face_cascade.empty():
    print(f"⚠️ Gagal memuat model Haarcascade dari: {cascade_path}")
else:
    print("✅ Haarcascade berhasil dimuat.")

✅ Haarcascade berhasil dimuat.


In [6]:
def register_user(user_id, num_images=5):
    user_folder = os.path.join(dataset_dir, user_id)
    os.makedirs(user_folder, exist_ok=True)

    cap = cv.VideoCapture(0)
    captured_images = 0

    while captured_images < num_images:
        ret, frame = cap.read()
        if not ret:
            break

        gray = cv.cvtColor(frame, cv.COLOR_BGR2GRAY)
        faces = face_cascade.detectMultiScale(gray, 1.3, 5)

        for (x, y, w, h) in faces:
            face = frame[y:y+h, x:x+w]
            face_resized = cv.resize(face, (160, 160))

            image_path = os.path.join(user_folder, f"{user_id}_front_{captured_images}.jpg")
            cv.imwrite(image_path, face_resized)
            captured_images += 1
            print(f"Captured {captured_images}/{num_images} images for {user_id}.")

        cv.putText(frame, "Position your face properly. Capturing...", (10, 30),
                   cv.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)
        cv.imshow('Register User', frame)
        cv.waitKey(1000)

    cv.putText(frame, "Press 'g' to capture with glasses or 'n' to skip", (10, 30),
               cv.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 255), 2)
    cv.imshow('Register User', frame)

    while True:
        key = cv.waitKey(0) & 0xFF
        if key == ord('g'):
            glasses_folder = os.path.join(user_folder, 'glasses')
            os.makedirs(glasses_folder, exist_ok=True)

            ret, frame = cap.read()
            if not ret:
                break

            gray = cv.cvtColor(frame, cv.COLOR_BGR2GRAY)
            faces = face_cascade.detectMultiScale(gray, 1.3, 5)

            for (x, y, w, h) in faces:
                face = frame[y:y+h, x:x+w]
                face_resized = cv.resize(face, (160, 160))

                image_path = os.path.join(glasses_folder, f"{user_id}_glasses.jpg")
                cv.imwrite(image_path, face_resized)
                print(f"Captured image with glasses for {user_id}.")
            break
        elif key == ord('n'):
            break

    cap.release()
    cv.destroyAllWindows()
    print(f"Registration complete for user {user_id}.")

In [7]:
# Registrasi pengguna baru
user_id = input("Enter your NIM or ID to register: ")
register_user(user_id)

Captured 1/5 images for 2110511131.
Captured 2/5 images for 2110511131.
Captured 3/5 images for 2110511131.
Captured 4/5 images for 2110511131.
Captured 5/5 images for 2110511131.
Registration complete for user 2110511131.


In [8]:
# Memuat dataset setelah registrasi
X, y = load_dataset(dataset_dir)
X = X.astype('float32') / 255.0

print(f"Total images: {X.shape[0]}, Image shape: {X.shape[1:]}")
print(f"Labels: {np.unique(y)}")

# Encoding label
label_names = np.unique(y)
label_map = {name: idx for idx, name in enumerate(label_names)}
y_encoded = np.array([label_map[label] for label in y])

# Membagi dataset
X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.2, random_state=42)
y_train = to_categorical(y_train, num_classes=len(label_names))
y_test = to_categorical(y_test, num_classes=len(label_names))

Scanning dataset directory: C:/Users/jefta/OneDrive/Dokumen/Tugas Akhir Next js/prototype-leads/Backend/dataset_model_skripsi
Processing NIM: 2110511131
Processing: C:/Users/jefta/OneDrive/Dokumen/Tugas Akhir Next js/prototype-leads/Backend/dataset_model_skripsi\2110511131\2110511131_front_0.jpg
Processing: C:/Users/jefta/OneDrive/Dokumen/Tugas Akhir Next js/prototype-leads/Backend/dataset_model_skripsi\2110511131\2110511131_front_1.jpg
Processing: C:/Users/jefta/OneDrive/Dokumen/Tugas Akhir Next js/prototype-leads/Backend/dataset_model_skripsi\2110511131\2110511131_front_2.jpg
Processing: C:/Users/jefta/OneDrive/Dokumen/Tugas Akhir Next js/prototype-leads/Backend/dataset_model_skripsi\2110511131\2110511131_front_3.jpg
Processing: C:/Users/jefta/OneDrive/Dokumen/Tugas Akhir Next js/prototype-leads/Backend/dataset_model_skripsi\2110511131\2110511131_front_4.jpg
✅ Dataset loaded: 5 images found
Sample X[0] shape: (160, 160, 3)
Sample y[0]: 2110511131
Total images: 5, Image shape: (160, 1

In [9]:
# Fungsi untuk melatih model
def train_model():
    output_size = len(label_names)
    model = Sequential([
        Conv2D(16, (3, 3), padding='valid', activation='relu', input_shape=(160, 160, 3)),
        MaxPooling2D(pool_size=(2, 2)),
        Conv2D(32, (3, 3), padding='valid', activation='relu'),
        MaxPooling2D(pool_size=(2, 2)),
        Conv2D(32, (3, 3), padding='valid', activation='relu'),
        MaxPooling2D(pool_size=(2, 2)),
        Conv2D(32, (3, 3), activation='relu'),
        MaxPooling2D(pool_size=(2, 2)),
        Conv2D(64, (3, 3), activation='relu'),
        MaxPooling2D(pool_size=(2, 2)),
        Flatten(),
        Dense(output_size, activation='softmax')
    ])

    model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])
    model.fit(X_train, y_train, epochs=20, validation_data=(X_test, y_test))

    model.save('my_model_part_3.h5')
    print("Model trained and saved.")

In [10]:
# Latih model
train_model()

Epoch 1/20


c:\Users\jefta\OneDrive\Dokumen\Tugas Akhir Next js\prototype-leads\Backend\.venv\lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
c:\Users\jefta\OneDrive\Dokumen\Tugas Akhir Next js\prototype-leads\Backend\.venv\lib\site-packages\keras\src\ops\nn.py:908: UserWarning: You are using a softmax over axis -1 of a tensor of shape (None, 1). This axis has size 1. The softmax operation will always return the value 1, which is likely not what you intended. Did you mean to use a sigmoid instead?
  warnings.warn(
c:\Users\jefta\OneDrive\Dokumen\Tugas Akhir Next js\prototype-leads\Backend\.venv\lib\site-packages\keras\src\losses\losses.py:33: SyntaxWarning: In loss categorical_crossentropy, expected y_pred.shape to be (batch_si

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step - accuracy: 1.0000 - loss: 0.0000e+00 - val_accuracy: 1.0000 - val_loss: 0.0000e+00
Epoch 2/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 81ms/step - accuracy: 1.0000 - loss: 0.0000e+00 - val_accuracy: 1.0000 - val_loss: 0.0000e+00
Epoch 3/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 82ms/step - accuracy: 1.0000 - loss: 0.0000e+00 - val_accuracy: 1.0000 - val_loss: 0.0000e+00
Epoch 4/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 83ms/step - accuracy: 1.0000 - loss: 0.0000e+00 - val_accuracy: 1.0000 - val_loss: 0.0000e+00
Epoch 5/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 82ms/step - accuracy: 1.0000 - loss: 0.0000e+00 - val_accuracy: 1.0000 - val_loss: 0.0000e+00
Epoch 6/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 81ms/step - accuracy: 1.0000 - loss: 0.0000e+00 - val_accuracy: 1.0000 - val_loss: 0.0000e+00
Epoch 7/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 80ms/step - accuracy: 1.0000 - loss: 0.0000e+00 - val_accuracy: 1.0000 - val_loss: 0.0000e+00
Epoch 8/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 80ms/step - accuracy: 1.0000 - loss: 0.0000

Model trained and saved.


In [11]:
# Fungsi untuk mendeteksi gerakan wajah
def detect_face_motion(prev_face_position, current_face_position, threshold=20):
    if prev_face_position is None:
        return False
    distance_moved = np.linalg.norm(np.array(current_face_position) - np.array(prev_face_position))
    return distance_moved > threshold


In [12]:
# Memuat model yang telah dilatih
model = load_model("my_model_part_3.h5")
print("[INFO] Finish load model...")


[INFO] Finish load model...


In [ ]:
# Inisialisasi video capture
cap = cv.VideoCapture(0)
prev_time = 0
fps = 0
prev_face_position = None

while cap.isOpened():
    ret, frame = cap.read()
    if ret:
        curr_time = time.time()
        fps = 1 / (curr_time - prev_time)
        prev_time = curr_time

        gray = cv.cvtColor(frame, cv.COLOR_BGR2GRAY)
        faces = detector.detect_faces(frame)

        for result in faces:
            x, y, width, height = result['box']
            face_roi = frame[y:y + height, x:x + width]
            face_rgb = cv.cvtColor(face_roi, cv.COLOR_BGR2RGB)

            current_face_position = (x + width // 2, y + height // 2)

            if detect_face_motion(prev_face_position, current_face_position):
                print("Face moved!")

            prev_face_position = current_face_position

            face_img = cv.resize(face_roi, (160, 160))
            face_img = face_img / 255.0
            face_img = face_img.reshape(1, 160, 160, 3)

            result = model.predict(face_img)
            idx = result.argmax(axis=1)[0]
            confidence = result.max(axis=1)[0] * 100

            label_text = "%s (%.2f %%)" % (label_names[idx], confidence) if confidence > 70 else "N/A"
            cv.rectangle(frame, (x, y), (x + width, y + height), (0, 255, 0), 2)
            cv.putText(frame, label_text, (x, y - 10), cv.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 2)

        fps_text = "FPS: %.2f" % fps
        cv.putText(frame, fps_text, (10, 30), cv.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
        cv.imshow('Detect Face', frame)
    else:
        break

    if cv.waitKey(10) == ord('q'):
        break

cap.release()
cv.destroyAllWindows()

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 67ms/step


c:\Users\jefta\OneDrive\Dokumen\Tugas Akhir Next js\prototype-leads\Backend\.venv\lib\site-packages\keras\src\ops\nn.py:908: UserWarning: You are using a softmax over axis -1 of a tensor of shape (1, 1). This axis has size 1. The softmax operation will always return the value 1, which is likely not what you intended. Did you mean to use a sigmoid instead?
  warnings.warn(


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━